### Load data into Google BigQuery to allow for SQL querying

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Get project ID from .env
PROJECT_ID = os.getenv("PROJECT_ID")
DATASET_NAME = "vanai_hackathons"
TABLE_NAME = "music_survey_data"

INPUT_CSV_FILE_PATH=f"../data/raw/{TABLE_NAME}.csv"
OUTPUT_JSON_SCHEMA_FILE_PATH=f"../data/{TABLE_NAME}_schema_for_bq.json"

* Test the connection to bq using the command line tool

In [2]:
!bq ls --project_id=bigquery-public-data worldpop

        tableId         Type    Labels       Time Partitioning               Clustered Fields          
 --------------------- ------- -------- --------------------------- ---------------------------------- 
  population_grid_1km   TABLE            DAY (field: last_updated)   geog, country_name, alpha_3_code  


In [3]:
!bq mk --dataset \
  --project_id=$PROJECT_ID \
  $PROJECT_ID:$DATASET_NAME

Dataset 'jas-test:vanai_hackathons' successfully created.


In [4]:
!pwd

/Users/jas/github/vanai-hackathon-004/scripts


In [14]:
!echo $PROJECT_ID:$DATASET_NAME.$TABLE_NAME 

jas-test:.


In [15]:
!echo $DATASET_NAME

vanai_hackathons


In [20]:
!echo $TABLE_NAME

music_survey_data


In [18]:
!echo $PROJECT_ID:$DATASET_NAME

jas-test:vanai_hackathons


In [24]:
!echo $PROJECT_ID:$DATASET_NAME.$TABLE_NAME

jas-test:.


In [25]:
!echo "${PROJECT_ID}:${DATASET_NAME}.${TABLE_NAME}"

-test:.


In [26]:
!echo $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME

jas-test:vanai_hackathons.music_survey_data


In [ ]:
# # Step 1: Load the CSV file into BigQuery and create a new table
# # The --autodetect flag tells BigQuery to automatically determine the schema
# # The --source_format=CSV specifies that the input is a CSV file
# !bq load \
#   --autodetect \
#   --source_format=CSV \
#   $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME \
#   ../data/raw/$TABLE_NAME"."csv

# # Step 2: Verify the table creation and schema
# # This command will show the table details, including the automatically detected schema
# !bq show $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME

Upload complete.
Waiting on bqjob_r376f6247d61cb246_000001997a2f5e14_1 ... (0s) Current status: DONE   
BigQuery error in load operation: Error processing job 'jas-
test:bqjob_r376f6247d61cb246_000001997a2f5e14_1': Error while reading data,
error message: CSV table encountered too many errors, giving up. Rows: 429;
errors: 100. Please look into the errors[] collection for more details.
Failure details:
- Error while reading data, error message: CSV processing encountered
too many errors, giving up. Rows: 429; errors: 100; max bad: 0;
error percent: 0
- Error while reading data, error message: Missing close quote
character (").; line_number: 2 byte_offset_to_start_of_line: 2043
column_index: 57 column_name: "Q19_Lyric_that_st..." column_type:
STRING value: "Clarify, rather t..."
- Error while reading data, error message: Missing close quote
character (").; line_number: 25 byte_offset_to_start_of_line: 29065
column_index: 54 column_name: "Q18_Life_theme_song" column_type:
STRING value: "

* Automatic schema detection didn't work

1. **Prepare the Schema**: Since we want all columns to be strings, we can use a wildcard schema definition.

2. **Use the `bq load` Command**: We'll use the `bq load` command with appropriate flags to specify the source file, destination table, and schema.


In [ ]:
!bq load --source_format=CSV \
        --autodetect=false \
        --skip_leading_rows=1 \
        --max_bad_records=0 \
        --field_delimiter=',' \
        --quote="" \
        --schema='*' \
        $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME \
        ../data/raw/$TABLE_NAME"."csv

!bq show $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME

BigQuery error in load operation: Invalid field name "*". Fields must contain
the allowed characters, and be at most 300 characters long. For allowed
characters, please refer to
https://cloud.google.com/bigquery/docs/schemas#column_names
BigQuery error in show operation: Not found: Table jas-
test:vanai_hackathons.music_survey_data


In [33]:
import os
print(os.getcwd())

/Users/jas/github/vanai-hackathon-004/scripts


* Create custom code to create a json schema based on the data before uploading to BQ

In [35]:
import csv
import json
import re

def clean_column_name(name):
    # Replace invalid characters with underscore, start with letter or underscore, max 128 chars
    clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', name)
    if not re.match(r'^[a-zA-Z_]', clean_name):
        clean_name = '_' + clean_name
    return clean_name[:128]

def generate_schema_from_csv(csv_path, json_path):
    with open(csv_path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        columns = next(reader)  # first row - column names

    schema = []
    for col in columns:
        clean_col = clean_column_name(col)
        schema.append({"name": clean_col, "type": "STRING"})

    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(schema, f, indent=2)

    print(f"Schema JSON saved to {json_path}")

# Example usage:
csv_file_path = INPUT_CSV_FILE_PATH      # change to your csv file path
json_schema_path = OUTPUT_JSON_SCHEMA_FILE_PATH # change to save location for schema json

generate_schema_from_csv(csv_file_path, json_schema_path)


Schema JSON saved to ../data/music_survey_data_schema_for_bq.json


In [ ]:
!bq load --source_format=CSV \
    --skip_leading_rows=1 \
    --schema=$OUTPUT_JSON_SCHEMA_FILE_PATH \
    $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME \
    $INPUT_CSV_FILE_PATH

!bq show $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME

Upload complete.
Waiting on bqjob_r3ff8724fe45041a8_000001997a43e6bc_1 ... (0s) Current status: DONE   
BigQuery error in load operation: Error processing job 'jas-
test:bqjob_r3ff8724fe45041a8_000001997a43e6bc_1': Error while reading data,
error message: CSV table encountered too many errors, giving up. Rows: 262;
errors: 100. Please look into the errors[] collection for more details.
Failure details:
- Error while reading data, error message: CSV processing encountered
too many errors, giving up. Rows: 262; errors: 100; max bad: 0;
error percent: 0
- Error while reading data, error message: CSV table references
column position 85, but line contains only 58 columns.;
line_number: 2 byte_offset_to_start_of_line: 2043 column_index: 85
column_name: "YOBClosed" column_type: STRING
- Error while reading data, error message: CSV table references
column position 85, but line contains only 58 columns.;
line_number: 35 byte_offset_to_start_of_line: 38592 column_index:
85 column_name: "YOBClose

* CSV validator script needed

In [41]:
import csv

def validate_csv(file_path, delimiter=',', max_errors=20):
    errors = 0

    with open(file_path, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=delimiter)
        
        try:
            header = next(reader)
        except Exception as e:
            print(f"Error reading header: {e}")
            return
        
        expected_columns = len(header)
        print(f"Header columns: {expected_columns}")

        for line_num, row in enumerate(reader, start=2):
            # Check column count
            if len(row) != expected_columns:
                print(f"Error line {line_num}: Expected {expected_columns} columns, found {len(row)} columns.")
                errors += 1
            
            # Optional: check for unclosed quotes by attempting to join and parse the line again
            # This is tricky here, usually csv.reader handles quoting, so focus on column count primarily

            if errors >= max_errors:
                print(f"Max error count {max_errors} reached. Stopping validation.")
                break

    if errors == 0:
        print("CSV validation passed - all rows have consistent column counts.")
    else:
        print(f"CSV validation ended with {errors} errors detected.")

# Usage example:
csv_file = INPUT_CSV_FILE_PATH  # change to your CSV file path
validate_csv(csv_file)



Header columns: 86
CSV validation passed - all rows have consistent column counts.


In [ ]:
# --allow_quoted_newlines=true to allow newlines within quoted strings.
# --field_delimiter=',' explicitly set delimiter.

In [42]:
!bq load --source_format=CSV \
    --skip_leading_rows=1 \
    --schema=$OUTPUT_JSON_SCHEMA_FILE_PATH \
    --field_delimiter=',' \
    --allow_quoted_newlines=true \
    $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME \
    $INPUT_CSV_FILE_PATH

!bq show $PROJECT_ID:$DATASET_NAME"."$TABLE_NAME

Upload complete.
Waiting on bqjob_r15c3c3bdec5481df_000001997a493f3e_1 ... (1s) Current status: DONE   
Table jas-test:vanai_hackathons.music_survey_data

   Last modified                                 Schema                                 Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- ------------------------------------------------------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  23 Sep 22:54:08   |- participant_id: string                                           1006         1252577                                                           1252577                                              
                    |- distribution_type: string                                                                                                                                      